실전1  

시중 TV제품별 분석  
시중 판매중인 TV정보를 온라인 쇼핑몰 웹페이지에서 크롤링하여 분석을 하고자 한다. 해당 데이터를 기반으로 각 TV의 제원에 따른 평가는 어떠한지 알아보고 향후 신제품 출시에 참고하고자 한다.  

데이터: TV.csv  666 rows, 11 columns, UTF-8  

변수 상세  
변수명  유형  설명  
Product_name    string     상품명  
Stars   float  평가 평균점수  
Ratings   int    평가수  
Reviews  int  후기 수  
current_price  int  현재 가격  
MRP  int  공장출고가  
channel  string  프리미엄 채널 서비스 제공목록  
Operating_system  string  운영체제  
Picture_quality  string  해상도  
Speaker  string  스피커  
Frequency  string  주사율

필요 라이브러리 함수 및 클래스 목록  
from sklearn.ensemble import RandomForestRegressor  

문제1. 주사율은 TV제원 중 가격에 큰 영향을 미치는 제원이다. 그러데 데이터 수집 중 실수가 발생하여 주사율 값 중 일부가 해상도, 스피커 변수에 잘못 입력된 것이 발견되었다. 세 변수에 흩어져 있는 주사율 데이터를 취합하여 전체 TV중에서 주사율이 60Hz인 TV는 총 몇 대인가?  
. 주사율은 2~3자리 숫자뒤에 반드시 Hz라는 단위로 표기되어 있다.  
. 세 변수 모두 주사율 정보가 없을 경우 결측치로 간주하고 이를 분석에서 제외하시오.  

문제2. TV의 해상도는 HD, 4K, 8K로 나뉜다. 최근 시장에 등장한 8K제품군은 상대적으로 비싸 비교적 매출이 떨어지는 편이다. 하지만 8K제품군 구매자가 4K제품군 구매자 대비 얼마나 만족하고 있는지 확인해보고자 한다. 8K제품군의 평가평균점수(Stars)의 평균값과 4K제품군의 평가평균점수와의 평균값 차이의 절대값을 산출하시오.  
. 화면 해상도는 관련 변수에서 "HD", "4K", "8K"로 찾아낼 수 있다.  
. 화면해상도 정보는 'Operating_system', 'channel', 'Picture_quality'변수에 흩어져 있다.  
. 결과는 반올림하여 소수점 2째자리까지 출력하시오

문제3. 좋은 평가를 받는 제품의 조건을 알아보고자 한다. 이를 위해 기존 변수와 더불어 여러 파생변수를 생성 후 해당 변수를 독립변수로 하고 평가 평균점수(Stars)를 종속변수로 하여 Random Forest 분석을 통해 좋은 평가를 받는 제품의 조건을 알아보고자 한다. 아래 절차를 수행하여 변수 중요도를 확인하고, 그 값 중 가장 큰 값을 가진 변수명을 기술하시오.  
. < 독립변수 >  
- 후기 작성비율: 후기수(Reviews)/평가수(Ratings)  
- 공장 출고가 (MRP)  
- 할인율: 현재 판매가 (current_price)/공장출고가 (MRP)  
- Neflix 제공여부: 제공(1), 미제공(0)  
- Prime Video제공여부 제공(1), 미제공(0)  
- 고해상도여부 4K 또는 8K (1), 나머지 (0)  
. < 종속변수>  
- 평가 평균점수 (Stars)  

. channel 변수에 해상도('Pixel'문자 참조) 또는 운영제체 정보('Oper'문자 참조)가 있는 TV는 분석에서 제외하시오.  
. Netflix, Prime Video 제공여부변수는 channel 변수를 참고하시오.  
. 독립변수를 생성하면서 결측치가 생성되는 경우 해당 행을 모두 제거하시오.  
. 학습 대상이 되는 행 개수는 197이다.  
. 고해상도 여부 변수 생성은 해상도변수만 참고하여 생성하시오  
. seed는 123 으로 지정하시오  
(정답예시: 고해상도여부 - 띄어쓰지 않고 문제에서 제시된 독립변수 명을 기입)

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv("TV.csv")
df.head(2)

,Product_Name,Stars,Ratings,Reviews,current_price,MRP,channel,Operating_system,Picture_quality,Speaker,Frequency
0,Croma,4.2,1773,217,7990,20000,HD Ready 1366 x 768 Pixels,20 Speaker Output,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty
1,Adsun,3.8,6742,930,8699,21999,Netflix|Disney+Hotstar|Youtube,Operating System: Android Based,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate


### Q1.

In [3]:
df_q1 = df.loc[:, "Picture_quality":].copy()
df_q1.head(2)

,Picture_quality,Speaker,Frequency
0,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty
1,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate


In [5]:
stat_1 = sum(df_q1["Picture_quality"] == "60 Hz Refresh Rate")
stat_2 = sum(df_q1["Speaker"] == "60 Hz Refresh Rate")
stat_3 = sum(df_q1["Frequency"] == "60 Hz Refresh Rate")
stat_1 + stat_2 + stat_3

509

In [13]:
# df_q1["Picture_quality"].unique()
ser_u = pd.Series(df_q1["Picture_quality"].unique())
# ser_u = pd.Series(df_q1["Speaker"].unique())
# ser_u = pd.Series(df_q1["Frequency"].unique())
# ser_u[ser_u.str.contains("Hz")]
ser_u[ser_u.str.contains("[0-9]{2,3} Hz")]

0                                    60 Hz Refresh Rate
15                                  120 Hz Refresh Rate
16                                  100 Hz Refresh Rate
18                                  200 Hz Refresh Rate
20                                   58 Hz Refresh Rate
22                                   50 Hz Refresh Rate
48    60 Hz Refresh Rate for a blur-free picture qua...
61                                  300 Hz Refresh Rate
dtype: object

In [14]:
df_q1["hz_cnt1"] = df_q1["Picture_quality"].str.contains("Hz") + 0
df_q1["hz_cnt2"] = df_q1["Speaker"        ].str.contains("Hz") + 0
df_q1["hz_cnt3"] = df_q1["Frequency"      ].str.contains("Hz") + 0
df_q1["hz_cnt" ] = df_q1["hz_cnt1"] + df_q1["hz_cnt2"] + df_q1["hz_cnt3"]
df_q1.head(2)

,Picture_quality,Speaker,Frequency,hz_cnt1,hz_cnt2,hz_cnt3,hz_cnt
0,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty,1,0,0,1
1,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0,0,1,1


In [15]:
df_q1["hz_cnt"].value_counts()

hz_cnt
1    662
0      4
Name: count, dtype: int64

In [17]:
df_q1_sub = df_q1.loc[df_q1["hz_cnt"] != 0, ]
len(df_q1_sub)

662

In [22]:
df_q1_sub.iloc[:, :3].apply(lambda x: x.str.contains("60 Hz").sum(), axis = 1).value_counts()

1    510
0    152
Name: count, dtype: int64

In [23]:
tx_regex = "^60 Hz|[^0-9]60 Hz" # 60 Hz로 시작하거나 60 Hz 앞에 숫자가 아닌 것이 위치하거나.
df_q1_sub.iloc[:, :3].apply(lambda x: x.str.contains(tx_regex).sum(), axis = 1).value_counts()

1    510
0    152
Name: count, dtype: int64

In [12]:
df_q1.apply(lambda x: x.str.contains("60 Hz").sum(), axis = 1).value_counts()[1]

510

### Q2.

In [36]:
df_q2 = df[["channel", "Operating_system", "Picture_quality", "Stars"]].copy()
df_q2.head(2)

,channel,Operating_system,Picture_quality,Stars
0,HD Ready 1366 x 768 Pixels,20 Speaker Output,60 Hz Refresh Rate,4.2
1,Netflix|Disney+Hotstar|Youtube,Operating System: Android Based,HD Ready 1366 x 768 Pixels,3.8


In [38]:
# ser_u = pd.Series(df_q2["channel"].unique())
# ser_u = pd.Series(df_q2["Operating_system"].unique())
ser_u = pd.Series(df_q2["Picture_quality"].unique())
ser_u[ser_u.str.contains("4K|8K")]

In [39]:
df_q2["cnt_4k"] = df_q2.iloc[:, :3].apply(lambda x: x.str.contains("4K").sum(), axis = 1)
df_q2["cnt_8k"] = df_q2.iloc[:, :3].apply(lambda x: x.str.contains("8K").sum(), axis = 1)
df_q2.head(3)

,channel,Operating_system,Picture_quality,Stars,cnt_4k,cnt_8k
0,HD Ready 1366 x 768 Pixels,20 Speaker Output,60 Hz Refresh Rate,4.2,0,0
1,Netflix|Disney+Hotstar|Youtube,Operating System: Android Based,HD Ready 1366 x 768 Pixels,3.8,0,0
2,Netflix|Prime Video|Disney+Hotstar|Youtube,Operating System: WebOS,HD Ready 1366 x 768 Pixels,4.4,0,0


In [43]:
df_q2["cnt_4k"].value_counts()

In [44]:
df_q2["cnt_8k"].value_counts()

In [40]:
stat_4k = df_q2.loc[df_q2["cnt_4k"] >= 1, "Stars"].mean()
stat_8k = df_q2.loc[df_q2["cnt_8k"] >= 1, "Stars"].mean()
stat_4k, stat_8k

(3.2180821917808218, 3.6)

In [50]:
abs(stat_4k - stat_8k) # 이정도만 보여도 충분.

0.3819178082191783

In [41]:
round(abs(stat_4k - stat_8k), 2)

0.38

In [48]:
x = df_q2.iloc[0, :3]
# x.str.contains("4K")
x.str.contains("4K").sum()

0

In [49]:
df_q2.iloc[:, :3].apply(lambda x: x.str.contains("4K"), axis = 1)

### Q3.

In [54]:
df_q3 = df.loc[~df["channel"].str.contains("Pixel|Oper"), ].reset_index(drop = True)
len(df_q3)

548

In [52]:
df_q3.head(1)

,Product_Name,Stars,Ratings,Reviews,current_price,MRP,channel,Operating_system,Picture_quality,Speaker,Frequency
1,Adsun,3.8,6742,930,8699,21999,Netflix|Disney+Hotstar|Youtube,Operating System: Android Based,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate


In [55]:
df_q3["x1"] = df_q3["Reviews"] / df_q3["Ratings"]
df_q3["x2"] = df_q3["MRP"]
df_q3["x3"] = df_q3["current_price"] / df_q3["MRP"]
df_q3["x4"] = df_q3["channel"].str.contains("Netflix") + 0
df_q3["x5"] = df_q3["channel"].str.contains("Prime Video") + 0
df_q3["x6"] = df_q3["Picture_quality"].str.contains("4K|8K") + 0

In [56]:
df_q3.head(2)

,Product_Name,Stars,Ratings,Reviews,current_price,MRP,channel,Operating_system,Picture_quality,Speaker,Frequency,x1,x2,x3,x4,x5,x6
0,Adsun,3.8,6742,930,8699,21999,Netflix|Disney+Hotstar|Youtube,Operating System: Android Based,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0.137941,21999,0.395427,1,0,0
1,LG,4.4,38870,3443,16499,21990,Netflix|Prime Video|Disney+Hotstar|Youtube,Operating System: WebOS,HD Ready 1366 x 768 Pixels,10 W Speaker Output,50 Hz Refresh Rate,0.088577,21990,0.750296,1,1,0


In [57]:
df_model = df_q3[["Stars", "x1", "x2", "x3", "x4", "x5", "x6"]]
df_model.head(2)

,Stars,x1,x2,x3,x4,x5,x6
0,3.8,0.137941,21999,0.395427,1,0,0
1,4.4,0.088577,21990,0.750296,1,1,0


In [58]:
len(df_model)

548

In [60]:
df_model.isna().sum()

In [61]:
df_model = df_model.dropna()
df_model.head(2)

,Stars,x1,x2,x3,x4,x5,x6
0,3.8,0.137941,21999,0.395427,1,0,0
1,4.4,0.088577,21990,0.750296,1,1,0


In [62]:
df_model.isna().sum()

Stars    0
x1       0
x2       0
x3       0
x4       0
x5       0
x6       0
dtype: int64

In [63]:
len(df_model)

197

In [66]:
model_rf = RandomForestRegressor(random_state = 123)
model_rf.fit(X = df_model.drop(columns = "Stars"), 
             y = df_model["Stars"])
model_rf.feature_importances_

array([0.31220617, 0.17383963, 0.4446912 , 0.02714087, 0.02400442,
       0.01811773])

In [67]:
# sklearn 버전에 따라 다음의 결과가 조금(또는...많이) 다를 수 있음.
pd.Series(model_rf.feature_importances_, index = df_model.columns[1:])

x1    0.312206
x2    0.173840
x3    0.444691
x4    0.027141
x5    0.024004
x6    0.018118
dtype: float64

x3 - 할인율!!!